# Mangrove Biomass Estimation: Thor Heyerdahl Climate Park

Modern visualization using Plotly's MapLibre integration

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

## Configuration & Data Generation

In [ ]:
# Configuration
LOCATION = 'Thor Heyerdahl Climate Park, Myanmar'
CENTER_LON, CENTER_LAT = 94.75, 16.75
BOUNDS = {'west': 94.7, 'east': 94.8, 'south': 16.7, 'north': 16.8}
GRID_SIZE = 40

# Create grid
lons = np.linspace(BOUNDS['west'], BOUNDS['east'], GRID_SIZE)
lats = np.linspace(BOUNDS['south'], BOUNDS['north'], GRID_SIZE)
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Generate NDVI and biomass
np.random.seed(42)
distance = np.sqrt((lon_grid - CENTER_LON)**2 + (lat_grid - CENTER_LAT)**2)
ndvi = 0.75 * np.exp(-distance * 20) + 0.3
ndvi += 0.1 * np.sin(lon_grid * 200) * np.cos(lat_grid * 200)
ndvi += np.random.normal(0, 0.02, lon_grid.shape)
ndvi = np.clip(ndvi, 0.2, 0.9)

biomass = np.maximum(250.5 * ndvi - 75.2, 0)

## Modern Map Visualization (MapLibre)

In [ ]:
# Create DataFrame for plotting
df = pd.DataFrame({
    'lon': lon_grid.flatten(),
    'lat': lat_grid.flatten(),
    'biomass': biomass.flatten(),
    'ndvi': ndvi.flatten()
})

# Create scatter map with MapLibre
fig = go.Figure()

# Add scatter layer
fig.add_trace(go.Scattermap(
    lon=df['lon'],
    lat=df['lat'],
    mode='markers',
    marker=dict(
        size=12,
        color=df['biomass'],
        colorscale='Viridis',
        cmin=df['biomass'].min(),
        cmax=df['biomass'].max(),
        colorbar=dict(
            title='Biomass<br>(Mg/ha)',
            thickness=20,
            len=0.7,
            x=0.98,
            y=0.5,
            yanchor='middle'
        ),
        opacity=0.85,
        sizemode='diameter'
    ),
    text=[f'Biomass: {b:.1f} Mg/ha<br>NDVI: {n:.3f}' 
          for b, n in zip(df['biomass'], df['ndvi'])],
    hovertemplate='<b>%{text}</b><br>Lon: %{lon:.4f}<br>Lat: %{lat:.4f}<extra></extra>',
    name=''
))

# Update to MapLibre with satellite style
fig.update_layout(
    map=dict(
        style='satellite',
        center=dict(lon=CENTER_LON, lat=CENTER_LAT),
        zoom=11.5,
        pitch=0,
        bearing=0
    ),
    height=650,
    title=dict(
        text=f'🌿 Mangrove Biomass - {LOCATION}',
        x=0.5,
        xanchor='center',
        font=dict(size=22, color='white'),
        y=0.98
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    showlegend=False,
    paper_bgcolor='#1a1a1a',
    plot_bgcolor='#1a1a1a'
)

# Add statistics box
stats_text = f"""<b style='color:white'>Statistics</b><br>
<span style='color:white'>Mean: {df['biomass'].mean():.1f} Mg/ha<br>
Max: {df['biomass'].max():.1f} Mg/ha<br>
Std: {df['biomass'].std():.1f} Mg/ha<br>
Points: {len(df):,}</span>"""

fig.add_annotation(
    text=stats_text,
    xref='paper', yref='paper',
    x=0.02, y=0.98,
    showarrow=False,
    bgcolor='rgba(0,0,0,0.7)',
    bordercolor='white',
    borderwidth=1,
    font=dict(size=12, color='white'),
    align='left'
)

fig.show()

## Alternative: Density Heatmap (New Syntax)

In [ ]:
# Create density map using new syntax
fig = go.Figure()

# Add density layer (new MapLibre syntax)
fig.add_trace(go.Densitymap(
    lon=df['lon'],
    lat=df['lat'],
    z=df['biomass'],
    radius=15,
    colorscale='Viridis',
    showscale=True,
    colorbar=dict(
        title='Biomass<br>(Mg/ha)',
        thickness=20,
        len=0.7,
        x=0.98
    ),
    opacity=0.8,
    name='Biomass Density'
))

# Update with MapLibre layout
fig.update_layout(
    map=dict(
        style='satellite',
        center=dict(lon=CENTER_LON, lat=CENTER_LAT),
        zoom=11
    ),
    height=600,
    title='Mangrove Biomass Density Map',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

## Choropleth with Hexagonal Binning

In [ ]:
# Create hexbin aggregation
import math

def create_hexagon(center_lon, center_lat, size=0.002):
    """Create hexagon polygon coordinates"""
    angles = [i * 60 for i in range(6)]
    coords = []
    for angle in angles:
        rad = math.radians(angle)
        lon = center_lon + size * math.cos(rad)
        lat = center_lat + size * math.sin(rad) * 0.8  # Adjust for latitude
        coords.append([lon, lat])
    coords.append(coords[0])  # Close polygon
    return coords

# Create hexagonal grid
hex_size = 0.005
hex_data = []

for i in range(0, len(df), 20):  # Sample every 20th point
    row = df.iloc[i]
    hex_coords = create_hexagon(row['lon'], row['lat'], hex_size)
    hex_data.append({
        'coords': hex_coords,
        'biomass': row['biomass'],
        'center_lon': row['lon'],
        'center_lat': row['lat']
    })

# Create figure with hexagons
fig = go.Figure()

for hex_info in hex_data:
    coords = hex_info['coords']
    lons = [c[0] for c in coords]
    lats = [c[1] for c in coords]
    
    # Determine color based on biomass
    norm_value = (hex_info['biomass'] - df['biomass'].min()) / (df['biomass'].max() - df['biomass'].min())
    color = px.colors.sample_colorscale('Viridis', [norm_value])[0]
    
    fig.add_trace(go.Scattermap(
        lon=lons + [lons[0]],
        lat=lats + [lats[0]],
        mode='lines',
        fill='toself',
        fillcolor=color,
        line=dict(width=0.5, color='white'),
        opacity=0.7,
        hovertemplate=f'Biomass: {hex_info["biomass"]:.1f} Mg/ha<extra></extra>',
        showlegend=False
    ))

fig.update_layout(
    map=dict(
        style='satellite',
        center=dict(lon=CENTER_LON, lat=CENTER_LAT),
        zoom=11
    ),
    height=600,
    title='Hexagonal Biomass Aggregation',
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

## Interactive 3D Visualization

In [ ]:
# 3D surface with enhanced interactivity
fig = go.Figure(data=[go.Surface(
    x=lons,
    y=lats,
    z=biomass,
    colorscale='Viridis',
    colorbar=dict(
        title='Biomass<br>(Mg/ha)',
        thickness=20,
        len=0.5,
        x=1.02
    ),
    contours=dict(
        z=dict(
            show=True,
            usecolormap=True,
            highlightcolor='white',
            project=dict(z=True)
        )
    ),
    hovertemplate='<b>Biomass: %{z:.1f} Mg/ha</b><br>Lon: %{x:.4f}<br>Lat: %{y:.4f}<extra></extra>'
)])

fig.update_layout(
    title='3D Biomass Surface with Contours',
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude', 
        zaxis_title='Biomass (Mg/ha)',
        camera=dict(
            eye=dict(x=1.8, y=-1.8, z=1.5),
            center=dict(x=0, y=0, z=-0.1)
        ),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.5)
    ),
    height=600,
    template='plotly_dark'
)

fig.show()

## Comprehensive Dashboard

In [ ]:
# Create comprehensive dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        'Spatial Distribution',
        'Biomass Histogram', 
        'NDVI vs Biomass',
        'Latitude Gradient',
        'Longitude Gradient',
        'Biomass Classes'
    ),
    specs=[
        [{'type': 'heatmap'}, {'type': 'histogram'}, {'type': 'scatter'}],
        [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'pie'}]
    ],
    horizontal_spacing=0.12,
    vertical_spacing=0.15
)

# 1. Spatial heatmap
fig.add_trace(
    go.Heatmap(
        x=lons, y=lats, z=biomass,
        colorscale='Viridis',
        showscale=False
    ),
    row=1, col=1
)

# 2. Histogram
fig.add_trace(
    go.Histogram(
        x=df['biomass'],
        nbinsx=25,
        marker_color='green'
    ),
    row=1, col=2
)

# 3. NDVI-Biomass scatter
fig.add_trace(
    go.Scatter(
        x=df['ndvi'], y=df['biomass'],
        mode='markers',
        marker=dict(size=3, color=df['biomass'], colorscale='Viridis', showscale=False),
        opacity=0.5
    ),
    row=1, col=3
)

# 4. Latitude profile
lat_profile = df.groupby(pd.cut(df['lat'], 20))['biomass'].mean().reset_index()
lat_profile['lat_mid'] = lat_profile['lat'].apply(lambda x: x.mid)
fig.add_trace(
    go.Scatter(
        x=lat_profile['lat_mid'], y=lat_profile['biomass'],
        mode='lines+markers',
        line=dict(color='darkgreen', width=2)
    ),
    row=2, col=1
)

# 5. Longitude profile
lon_profile = df.groupby(pd.cut(df['lon'], 20))['biomass'].mean().reset_index()
lon_profile['lon_mid'] = lon_profile['lon'].apply(lambda x: x.mid)
fig.add_trace(
    go.Scatter(
        x=lon_profile['lon_mid'], y=lon_profile['biomass'],
        mode='lines+markers',
        line=dict(color='darkgreen', width=2)
    ),
    row=2, col=2
)

# 6. Biomass classes pie chart
biomass_classes = pd.cut(df['biomass'], 
                         bins=[0, 50, 100, 150, 200],
                         labels=['Low (<50)', 'Medium (50-100)', 'High (100-150)', 'Very High (>150)'])
class_counts = biomass_classes.value_counts()
fig.add_trace(
    go.Pie(
        labels=class_counts.index,
        values=class_counts.values,
        marker=dict(colors=['#fde725', '#5ec962', '#21918c', '#3b528b'])
    ),
    row=2, col=3
)

# Update layout
fig.update_layout(
    height=700,
    title_text='Mangrove Biomass Analysis Dashboard',
    showlegend=False,
    template='plotly_white'
)

# Update axes
fig.update_xaxes(title_text='Longitude', row=1, col=1)
fig.update_yaxes(title_text='Latitude', row=1, col=1)
fig.update_xaxes(title_text='Biomass (Mg/ha)', row=1, col=2)
fig.update_xaxes(title_text='NDVI', row=1, col=3)
fig.update_yaxes(title_text='Biomass (Mg/ha)', row=1, col=3)
fig.update_xaxes(title_text='Latitude', row=2, col=1)
fig.update_xaxes(title_text='Longitude', row=2, col=2)
fig.update_yaxes(title_text='Mean Biomass', row=2, col=1)
fig.update_yaxes(title_text='Mean Biomass', row=2, col=2)

fig.show()

## Summary Statistics & Carbon Assessment

In [ ]:
# Calculate comprehensive statistics
total_area_ha = 11000  # Approximate area in hectares
total_biomass = df['biomass'].mean() * total_area_ha
carbon_stock = total_biomass * 0.47  # Carbon fraction for mangroves
co2_equivalent = carbon_stock * 3.67  # CO2 to C ratio

# Create summary DataFrame
summary = pd.DataFrame([
    ['Location', LOCATION],
    ['Coordinates', f'{CENTER_LAT:.3f}°N, {CENTER_LON:.3f}°E'],
    ['Analysis Points', f'{len(df):,}'],
    ['Area Coverage', f'{total_area_ha:,} hectares'],
    ['', ''],
    ['Mean Biomass', f'{df["biomass"].mean():.1f} Mg/ha'],
    ['Median Biomass', f'{df["biomass"].median():.1f} Mg/ha'],
    ['Max Biomass', f'{df["biomass"].max():.1f} Mg/ha'],
    ['Min Biomass', f'{df["biomass"].min():.1f} Mg/ha'],
    ['Std Deviation', f'{df["biomass"].std():.1f} Mg/ha'],
    ['', ''],
    ['Total Biomass', f'{total_biomass:,.0f} Mg'],
    ['Carbon Stock', f'{carbon_stock:,.0f} Mg C'],
    ['CO₂ Equivalent', f'{co2_equivalent:,.0f} Mg CO₂'],
    ['Carbon Credit Value*', f'${carbon_stock * 15:,.0f} USD'],
], columns=['Metric', 'Value'])

# Display styled table
styled = summary.style.set_properties(**{
    'background-color': '#f8f9fa',
    'color': '#2c3e50',
    'border': '1px solid #dee2e6',
    'padding': '12px',
    'text-align': 'left',
    'font-size': '14px'
}).set_table_styles([
    {'selector': 'th',
     'props': [('background-color', '#2c3e50'),
               ('color', 'white'),
               ('font-weight', 'bold'),
               ('text-align', 'center'),
               ('padding', '12px'),
               ('font-size', '16px')]},
    {'selector': 'tr:hover',
     'props': [('background-color', '#e9ecef')]}
])

display(styled)
print('\n* Carbon credit value estimated at $15/tCO₂')